In [4]:
from graphdatascience import GraphDataScience

# Use Neo4j URI and credentials according to your setup
# NEO4J_URI could look similar to "bolt://my-server.neo4j.io:7687"
gds = GraphDataScience("bolt://localhost", auth=("neo4j","neo4j"))


# Check the installed GDS version on the server
print(gds.version())
assert gds.version()

2.2.5


In [5]:
# Create a minimal example graph
gds.run_cypher(
  """
  CREATE
  (m: City {name: "Malmö"}),
  (l: City {name: "London"}),
  (s: City {name: "San Mateo"}),
  (m)-[:FLY_TO]->(l),
  (l)-[:FLY_TO]->(m),
  (l)-[:FLY_TO]->(s),
  (s)-[:RUN_TO]->(l)
  """
)

# Project the graph into the GDS Graph Catalog
# We call the object representing the projected graph `G_office`
G_office2, project_result = gds.graph.project("neo4j-offices2", "City", "FLY_TO")



Loading: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 100.0/100 [00:00<00:00, 9375.68%/s]


In [6]:
# Run the mutate mode of the PageRank algorithm
mutate_result = gds.pageRank.mutate(G_office2, tolerance=0.5, mutateProperty="rank")

# We can inspect the node properties of our projected graph directly
# via the graph object and see that indeed the new property exists
assert G_office2.node_properties("City") == ["rank"]

In [7]:
mutate_result

mutateMillis                                                              0
nodePropertiesWritten                                                     3
ranIterations                                                             1
didConverge                                                            True
centralityDistribution    {'p99': 0.2774991989135742, 'min': 0.213749885...
postProcessingMillis                                                     20
preProcessingMillis                                                       0
computeMillis                                                             6
configuration             {'jobId': '2550ef70-ba30-4f06-9acf-88b0a769e4e...
Name: 0, dtype: object

In [8]:
gds.run_cypher(
  """
  CREATE
    (a:City {name: "New York City", settled: 1624}),
    (b:City {name: "Philadelphia", settled: 1682}),
    (c:City:Capital {name: "Washington D.C.", settled: 1790}),
    (d:City {name: "Baltimore"}),
    (e:City {name: "Atlantic City"}),
    (f:City {name: "Boston"}),

    (a)-[:ROAD {cost: 50}]->(b),
    (a)-[:ROAD {cost: 50}]->(c),
    (a)-[:ROAD {cost: 100}]->(d),
    (b)-[:ROAD {cost: 40}]->(d),
    (c)-[:ROAD {cost: 40}]->(d),
    (c)-[:ROAD {cost: 80}]->(e),
    (d)-[:ROAD {cost: 30}]->(e),
    (d)-[:ROAD {cost: 80}]->(f),
    (e)-[:ROAD {cost: 40}]->(f);
  """
)
G, project_result = gds.graph.project("road_graph", "City", {"ROAD": {"properties": ["cost"]}})

assert G.relationship_count() == 9

In [9]:
louvain_result = gds.louvain.mutate(
    G,                            #  Graph object
    maxIterations=5,              #  Configuration parameters
    mutateProperty="community"
)
assert louvain_result["communityCount"] > 0

fastrp_result = gds.fastRP.write(
    G,                                #  Graph object
    featureProperties=["community"],  #  Configuration parameters
    embeddingDimension=256,
    propertyRatio=0.3,
    relationshipWeightProperty="cost",
    writeProperty="embedding"
)
assert fastrp_result["nodePropertiesWritten"] == G.node_count()

In [10]:
fastrp_result

nodeCount                                                                9
nodePropertiesWritten                                                    9
preProcessingMillis                                                      0
computeMillis                                                            7
writeMillis                                                             11
configuration            {'writeConcurrency': 4, 'nodeSelfInfluence': 0...
Name: 0, dtype: object

In [11]:
source_id = gds.find_node_id(["City"], {"name": "New York City"})
target_id = gds.find_node_id(["City"], {"name": "Boston"})

result = gds.shortestPath.dijkstra.stream(
    G,
    sourceNode=source_id,
    targetNode=target_id,
    relationshipWeightProperty="cost"
)
assert result["totalCost"][0] == 160

In [12]:
result

,index,sourceNode,targetNode,totalCost,nodeIds,costs,path
0,0,3,8,160.0,"[3, 4, 6, 7, 8]","[0.0, 50.0, 90.0, 120.0, 160.0]","((cost), (cost), (cost), (cost))"


In [13]:
node1 = gds.find_node_id(["City"], {"name": "Boston"})
node2 = gds.find_node_id(["City"], {"name": "Atlantic City"})

score = gds.alpha.linkprediction.adamicAdar(node1, node2)
#assert round(score, 2) == 0.62


In [14]:
score

0.6213349345596119